In [1]:
import pandas as pd 
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
import itertools

from sklearn.model_selection import cross_val_score

In [2]:
df=pd.read_excel('JiraTicket_All_Stopword_Data_Removed_up_26-3-20.xlsx',sheet_name="Sheet1")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15628 entries, 0 to 15627
Data columns (total 9 columns):
Issue_key          15628 non-null object
Issue Type         15628 non-null object
Office Location    9517 non-null object
Pool               15628 non-null object
Service            15628 non-null object
Category           15628 non-null object
Summary            14853 non-null object
Description        15628 non-null object
Description1       15460 non-null object
dtypes: object(9)
memory usage: 1.1+ MB


In [4]:
df.dropna(subset=['Description1'],inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
df['Issue Type'].value_counts()

Service Request    8743
Incident           6717
Name: Issue Type, dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15460 entries, 0 to 15459
Data columns (total 9 columns):
Issue_key          15460 non-null object
Issue Type         15460 non-null object
Office Location    9442 non-null object
Pool               15460 non-null object
Service            15460 non-null object
Category           15460 non-null object
Summary            14689 non-null object
Description        15460 non-null object
Description1       15460 non-null object
dtypes: object(9)
memory usage: 1.1+ MB


In [7]:
df.head()

,Issue_key,Issue Type,Office Location,Pool,Service,Category,Summary,Description,Description1
0,IT-18397,Service Request,"San Jose, CA",sj-techstop,Personal Computing Hardware,Recover Hardware Assets,IT ticket to recover the hardware assets-vpu...,IT ticket to recover the hardware assets-vpuch...,ticket recover hardware assets vpuchin pc pc v...
1,IT-14182,Service Request,"Bangalore, India",India-pool,Personal Computing Hardware,New PC,3 of 5 - Please deploy Precision 3630 Tower C...,3 of 5 - Please deploy Precision 3630 Tower CT...,deploy precision tower cto base serial gybz as...
2,IT-37239,Service Request,"Petah Tikva, Israel",L2-Servicedesk,Vormetric,Vormetric Policy - Grant Access,access to ARM ASHBROOK,+underlined text+i need permission to acces AR...,underlined text need permission acces arm ashb...
3,IT-12754,Incident,"San Jose, CA",L2-Servicedesk,UNIX,VNC/NX issues,can't access certain file paths from my vnc,I can't access certain file paths from my vnc....,access certain file paths vnc icd check assist
4,IT-8901,Service Request,Noida,L2-Servicedesk,Hosting-Storage,Storage- Create Home directory,"create a directory named ""wangd"" under /grid/...","Please help to create a directory named ""wangd...",help create directory named wangd grid tfo scr...


In [ ]:
# df=df.sample(frac=1)

Linear SVC

In [8]:
pipeline = Pipeline([
        ('bow', CountVectorizer(ngram_range=(1,2))),  # strings to token integer counts
        ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
        #('Normalize', preprocessing.Normalizer()),
        ('c',LinearSVC()),  
    ])

In [9]:
X_train, X_test, y_train, y_test =  train_test_split(df['Description1'], df['Issue Type'], test_size=0.20, random_state=42)

In [ ]:
A_train, A_test, b_train, b_test =  train_test_split(df['Description'], df['Issue Type'], test_size=0.20, random_state=42)

In [ ]:
pipeline.fit(X_train,y_train)

In [ ]:
#Linear SVC
pred=pipeline.predict(X_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("Accuracy : ",accuracy_score(pred,y_test))

SVM

In [10]:
pipeline = Pipeline([
        ('bow', CountVectorizer(ngram_range=(1,2))),  # strings to token integer counts
        ('tfidf', TfidfTransformer()),  # inte/ger counts to weighted TF-IDF scores
        #('Normalize', preprocessing.Normalizer()),
        ('c',svm.SVC(kernel='linear',probability=True)),  
    ])

In [11]:
X_train, X_test, y_train, y_test =  train_test_split(df['Description1'], df['Issue Type'], test_size=0.20, random_state=42)

In [12]:
X_train=df["Description1"]
y_train=df["Issue Type"]

In [13]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_...ar', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [14]:
pred=pipeline.predict(X_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("Accuracy : ",accuracy_score(pred,y_test))

[[1324   63]
 [  83 1622]]
                 precision    recall  f1-score   support

       Incident       0.94      0.95      0.95      1387
Service Request       0.96      0.95      0.96      1705

      micro avg       0.95      0.95      0.95      3092
      macro avg       0.95      0.95      0.95      3092
   weighted avg       0.95      0.95      0.95      3092

Accuracy :  0.9527813712807245


Logistic_Regression

In [ ]:
pipeline = Pipeline([
        ('bow', CountVectorizer(ngram_range=(1,2))),  # strings to token integer counts
        ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
        #('Normalize', preprocessing.Normalizer()),
        ('c',LogisticRegression()),  
    ])

In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(df['Description1'], df['Issue Type'], test_size=0.20, random_state=42)

In [ ]:
pipeline.fit(X_train,y_train)

In [ ]:
pred=pipeline.predict(X_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("Accuracy : ",accuracy_score(pred,y_test))

In [15]:
import joblib
joblib.dump(pipeline,open('./Ticket_Type/All_SVM_Model_23_3-20.psm','wb'))

In [ ]:
# df=pd.DataFrame()

In [ ]:
# df['Description']=A_train
# df["Description1"]=X_train
# df["Issue Type"]=y_train


In [ ]:
# df['Description']=A_test
# df["Description1"]=X_test
# df["Issue Type"]=y_test


In [ ]:
# df.info()

In [ ]:
# writer = pd.ExcelWriter('Special_Stop_TestDataAE.xlsx')
# df.to_excel(writer,"Sheet1",index=False)
# writer.save()

In [ ]:
# testData=pd.read_excel("ResultTestAE_special_stop.xlsx")

In [ ]:
# testData["Result"] = testData["Issue Type"] == testData["Prediction_Ticket_Type"]

In [ ]:
# len(testData[testData["Result"] == True])/len(testData)

In [ ]:
# writer = pd.ExcelWriter('TestDataAE_special_stop_TF.xlsx')
# testData.to_excel(writer,"Sheet1",index=False)
# writer.save()

In [ ]:
prob =pipeline.predict_proba(X_test)

#Convert to list
y_list = y_test.values.tolist()
X_list = X_test.values.tolist()
A_list = A_test.values.tolist()

prob = prob.tolist()

In [ ]:
conf = [max(prob[i]) for i in range(len(prob))]

In [ ]:
d = { 'Prediction_Issue_Type': pred,'Actual_Issue_Type':y_list,'Original_Description':A_list,'Clean_Description':X_test,"Confidence":conf}
y_df = pd.DataFrame(d)
y_df.info()

In [ ]:
y_df

In [ ]:
y_df["Result"] = y_df["Actual_Issue_Type"] == y_df["Prediction_Issue_Type"]

In [ ]:
writer = pd.ExcelWriter('./Ticket_Type/TTS_Results_Issue_Type.xlsx')
y_df.to_excel(writer,"Sheet1")
writer.save()

In [ ]:
len(y_df[y_df["Result"] == True])/len(y_df)

In [ ]:
abovecf = y_df[y_df["Confidence"] >=0.1].copy()

In [ ]:
len(abovecf[abovecf["Result"] == True])/len(abovecf)

In [ ]:
allrecords = []
truerecords = []
falserecords = []
accuracy = []

In [ ]:
for i in range(0,10):
    i = i * 0.10
    abovecf = y_df[y_df["Confidence"] >= i].copy()
    allrecords.append(len(abovecf))
    truerecords.append(len(abovecf[abovecf["Result"] == True]))
    falserecords.append(len(abovecf[abovecf["Result"] == False]))
    accuracy.append(len(abovecf[abovecf["Result"] == True])/len(abovecf))

In [ ]:
d = {'Tickets count above confidence Threshold': allrecords,'Correctly predicted tickets': truerecords, 'Incorrectly  predicted tickets': falserecords ,'Accuracy after Threshhold':accuracy,'Threeshold' : [x * 0.1 for x in range(0, 10)]}
conf_data = pd.DataFrame(d)
conf_data.info()

In [ ]:
conf_data["Overall Tickets for Testing"] = len(y_df)

In [ ]:
conf_data["Tickets count routed  to manual Queue"] =conf_data["Overall Tickets for Testing"] - conf_data["Tickets count above confidence Threshold"]

In [ ]:
writer = pd.ExcelWriter('./Ticket_Type/TTS_Result_confidence.xlsx')
conf_data.to_excel(writer,"Sheet1")
writer.save()

In [ ]:
cm=confusion_matrix(y_test,pred)

In [ ]:
cr=classification_report(y_test,pred)

In [ ]:
file=open("./Ticket_Type/TTS_Classification_Report.xls","w")

In [ ]:
file.writelines([cr])

In [ ]:
a=pd.DataFrame(cm ,index=pipeline.classes_,columns=pipeline.classes_)

In [ ]:
a.to_excel("./Ticket_Type/TTS_Confusion_matrix.xlsx")

# Plugin Result on seperate Test Data

In [ ]:
testdata=pd.read_excel("./TestData/ResultTestAE_special_stop_Result.xlsx")

In [ ]:
testdata["Result"] = testdata["Issue Type"] == testdata["Prediction_Ticket_Type"]

In [ ]:
len(testdata[testdata["Result"] == True])/len(testdata)

In [ ]:
writer = pd.ExcelWriter('./TestData/Results_Ticket_Type.xlsx')
testdata.to_excel(writer,"Sheet1", index=False)
writer.save()

In [ ]:
abovecf = testdata[testdata["Prediction_Ticket_Type_confidence"] >=0.1].copy()

In [ ]:
len(abovecf[abovecf["Result"] == True])/len(abovecf)

In [ ]:
allrecords = []
truerecords = []
falserecords = []
accuracy = []

In [ ]:
for i in range(0,10):
    i = i * 0.10
    abovecf = testdata[testdata["Prediction_Ticket_Type_confidence"] >= i].copy()
    allrecords.append(len(abovecf))
    truerecords.append(len(abovecf[abovecf["Result"] == True]))
    falserecords.append(len(abovecf[abovecf["Result"] == False]))
    accuracy.append(len(abovecf[abovecf["Result"] == True])/len(abovecf))

In [ ]:
d = {'Tickets count above confidence Threshold': allrecords,'Correctly predicted tickets': truerecords, 'Incorrectly  predicted tickets': falserecords ,'Accuracy after Threshhold':accuracy,'Threeshold' : [x * 0.1 for x in range(0, 10)]}
conf_data = pd.DataFrame(d)
conf_data.info()

In [ ]:
conf_data["Overall Tickets for Testing"] = len(testdata)

In [ ]:
conf_data["Tickets count routed  to manual Queue"] =conf_data["Overall Tickets for Testing"] - conf_data["Tickets count above confidence Threshold"]

In [ ]:
writer = pd.ExcelWriter('./TestData/Result_confidence.xlsx')
conf_data.to_excel(writer,"Sheet1")
writer.save()

In [ ]:
df

In [ ]:
writer = pd.ExcelWriter('RandomTestData.xlsx')
df.to_excel(writer,"Sheet1",index=False)
writer.save()